In [1]:
import pandas as pd
import numpy as np

# Create the dataset
data = {
    'High School GPA': [3.8, 3.2, 4.0, 2.5, 3.6],
    'SAT Score': [1450, 1300, 1500, 1200, 1400],
    'Extracurricular Activities': ['Yes', 'No', 'Yes', 'Yes', 'No'],
    'Recommendation Letter Strength': ['Strong', 'Weak', 'Strong', 'Weak', 'Strong'],
    'Admission Status': ['Admitted', 'Not Admitted', 'Admitted', 'Not Admitted', 'Admitted']
}
df = pd.DataFrame(data)


In [2]:
# Define the entropy function
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True)
    entropy = np.sum([(-counts[i]/np.sum(counts)) * np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

# Define the information gain function
def InfoGain(data, split_attribute_name, target_name="Admission Status"):
    total_entropy = entropy(data[target_name])
    vals, counts = np.unique(data[split_attribute_name], return_counts=True)
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts)) * entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

# Define the ID3 algorithm
def ID3(data, original_data, features, target_attribute_name="Admission Status", parent_node_class=None):
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    elif len(data)==0:
        return np.unique(original_data[target_attribute_name])[np.argmax(np.unique(original_data[target_attribute_name],return_counts=True)[1])]
    elif len(features) == 0:
        return parent_node_class
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        item_values = [InfoGain(data, feature, target_attribute_name) for feature in features]
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        tree = {best_feature:{}}
        features = [i for i in features if i != best_feature]
        for value in np.unique(data[best_feature]):
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = ID3(sub_data,df,features,target_attribute_name,parent_node_class)
            tree[best_feature][value] = subtree
        return(tree)



In [3]:
# Train the decision tree
features = df.columns[:-1]
tree = ID3(df,df,features)

# Print the decision tree
def print_tree(tree, level=0):
    for split_rule, sub_tree in tree.items():
        if isinstance(sub_tree, dict):
            print('  ' * level + split_rule)
            print_tree(sub_tree, level + 1)
        else:
            print('  ' * level + split_rule + ':', sub_tree)

print_tree(tree)

# Define the predict function
def predict(query, tree, default='Admitted'):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]]
            except:
                return default
            if isinstance(result, dict):
                return predict(query, result)
            else:
                return result


High School GPA


TypeError: can only concatenate str (not "numpy.float64") to str